# Кредитный скоринг

Существует предположение:

* семейное положение и наличие детей влияет на на факт погашения кредита в срок.

По данным кредитного отдела банка проверим действительно ли такая взаимосвязь имеет место. Результаты исследования будут учтены при построении модели кредитного скоринга.

**Цель исследования** — ответить на четыре вопроса:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Описание данных:**

Данные о платежеспособности клиентов находятся в файле `data.csv`. О качестве данных ничего не известно. Поэтому перед началом анализа понадобится обзор данных. 

**Содержание исследования**:
 1. Обзор данных.
 2. Предобработка данных.
    * 2.1 Аномальные значения
    * 2.2 Пропуски значений
    * 2.3 Изменение типов данных
    * 2.4 Дубликаты
    * 2.5 Категоризация данных
 3. Ответы на вопросы.
 4. Итоги исследования

## 1. Обзор данных

Импортируем библиотеку `pandas` для работы с данными и читаем файл `data.csv`.

In [1]:
# импортируем библиотеку pandas и открываем файл с данными
import pandas as pd

try:
    df = pd.read_csv('C:\\Users\\Anna\\Desktop\\Python\\Yandex Praktikum\\Datasets\\project_2\\data.csv')
except:
    df = pd.read_csv('/datasets/data.csv')

Выводим первые 10 строк таблицы, а также общую информацию о ней.

In [2]:
# выводим первые 10 строк таблицы и сводную информацию о ней
display(df.head(10))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов, тип данных в таблице - `int64`, `float64` и `object`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения. В колонке с показателем стажа `days_employed` имеются отрицательные значения, а также аномально завышенные значения, что очевидно является ошибкой. В колонке `education` встречаются неявные дубликаты. А по столбцу `total_income` используется тип данных `float`, его следует изменить на `int`.

**Выводы**
    
В каждой строке таблицы — данные о клиенте. Большая часть колонок характеризует самого клиента: количество детей, возраст, семейное положение, образование, стаж и доход, а также цель получения кредита. Есть колонки, которые присваивают отдельный идентификатор определенным характеристикам клиента. Например, наличие высшего образования обозначается цифрой 0, среднего - 1.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, дубликаты, а также аномльные значения.

Чтобы двигаться дальше, нужно устранить проблемы в данных.
</div>

## 2. Предобработка данных

### 2.1 Аномальные значения

Сначала обработаем аномальные значения, так как при дальнейшей обработке данных (например, заполнении пропусков средними значениями), такие аномалии повлияют на результат.

По столбцу с данными о стаже `days_employed` есть отрицательные значения. Предположим, что это техническая ошибка и такие значения должны быть взяты по модулю.

In [3]:
# убираем отрицательные значения по столбцу 'days_employed'
#df.loc[df['days_employed'] < 0, 'days_employed'] = df.loc[df['days_employed'] < 0, 'days_employed'] *-1
df.loc[df['days_employed'] < 0, 'days_employed'] = abs(df.loc[df['days_employed'] < 0, 'days_employed'])

Проверим наличие отрицательных значений по фильтру.

In [4]:
# выводим столбец 'days_employed' только по отрицательным значениям
df.loc[df['days_employed'] < 0, 'days_employed']

Series([], Name: days_employed, dtype: float64)

Фильтр возвращат пустой объект, значит отрицательных значений по столбцу `days_employed` не осталось.

---

В столбце со стажем `days_employed` есть очевидно зывышенные значения. Определим ***максимально возможный стаж*** как ***максимальный возраст заемщика*** - ***минимальный возраст для начала трудовой деятельности (16 лет)***. Проверим, какую долю в датафрейме занимают значения, превышающие максимально возможное.

In [5]:
# находим максимально возможный стаж в днях
max_days_employed = (df['dob_years'].max() - 16) * 365

# фильтруем датафрейм и найдем долю аномальных значений в общем числе
df_filtred = df[df['days_employed'] > max_days_employed]
days_employed_anomal_count = df_filtred['days_employed'].count()
days_employed_count = df['days_employed'].count()
days_employed_anomal_ratio = days_employed_anomal_count / days_employed_count
print('Доля аномальных значений в стаже составляет {:.0%}'.format(days_employed_anomal_ratio))

Доля аномальных значений в стаже составляет 18%


Так как у нас нет информации о природе этой аномалии, мы не можем исправить эти значения на корректные. 18% - слишком большая доля для того, чтобы просто отбросить эти значения, поэтому оставим их в датафрейме. 

Предположим, что аномально большие значения введены в часах, а не в днях. Тогда переведем эти значения в дни.

Однако на наше исследование эти данные не повлияют. В иных условиях в таком случае следует обращаться к источнику данных за дополнительной информацией.

In [6]:
# перевод аномально больших значений из часов в дни
df.loc[df['days_employed'] > max_days_employed, 'days_employed'] = df.loc[df['days_employed'] > max_days_employed, 'days_employed'] / 24

In [7]:
# выводим столбец 'days_employed' по аномально большим значениям
df.loc[df['days_employed'] > max_days_employed, 'days_employed']

Series([], Name: days_employed, dtype: float64)

Фильтр возвращат пустой объект, значит аномальных значений по столбцу `days_employed` не осталось.

---

Проверим корректность данных о возрасте заемщиков по столбцу `dob_years`. Для этого выведем все встречающиеся значения возраста.

In [8]:
# выводим уникальные значения по столбцу 'dob_years'
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

Среди значений возраста есть аномальное значение 0. Вероятнее всего это техническая ошибка. Заменим нулевые значения на медианные через функцию замены некорректных значений.

Функция `replace_wrong_values()` принимает на вход 3 параметра:
* `col` — наименование столбца, в котором будет производиться замена,
* `wrong_values` — список некорректных значений,
* `correct_value` — строка с правильным значением.

Функция должна исправить колонку `col` в таблице `df`: заменить каждое значение из списка `wrong_values` на значение из `correct_value`.

In [9]:
# функция для замены некорректных значений
def replace_wrong_values(col, wrong_values, correct_value):
    for value in wrong_values:
        df[col] = df[col].replace(value, correct_value)

Применим функцию `replace_wrong_values()` и передим ей аргументы для замены значения `0` на медианное значение `dob_years_median` в столбце `dob_years`.

In [10]:
# находим медианное значение по столбцу 'days_employed'
dob_years_median = df['dob_years'].median()

# применяем функцию 'replace_wrong_values'
replace_wrong_values('dob_years', [0], dob_years_median)

Проверим встречающиеся значения возраста после исправления.

In [11]:
# выводим уникальные значения по столбцу 'dob_years'
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

Аномальных значений нет, исправление произведено корректно.

---

Проверим аномальные значения по столбцу `children`. Для этого выведем все уникальные значения.

In [12]:
# выводим уникальные значения по столбцу 'children'
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

По столбцу `children` имеется два аномальных значения:
1. Аномальное значение 20 - предположим, что при вводе данных была допущена ошибка и добавлен лишний 0. Исправим значения 20 на 2.

2. Отрицательное значение -1 - предположим, что это техническая ошибка и следует взять число по модулю.

Исправим обе аномалии с помощью функции `replace_wrong_values`.

In [13]:
# применяем функцию 'replace_wrong_values'      
replace_wrong_values('children', [20], 2)
replace_wrong_values('children', [-1], 1)

Проверим уникальные значения возраста после исправления.

In [14]:
# выводим уникальные значения по столбцу 'children'
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Аномальных значений не осталось, можно приступать к пропускам значений.

### 2.2 Пропуски значений

Теперь перейдем к пропускам. Сначала посчитаем сколько в таблице пропущенных значений.

In [15]:
# подсчет пропущенных значений
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В данных пропущены значения в столбцах `days_employed` и `total_income`. Проверим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками.

In [16]:
# доля пропущенных значений в столбце 'days_employed' от общего числа значений
null_ratio = df['days_employed'].isna().sum() / df.shape[0]
print('Доля пропусков в столбце days_employed составляет {:.0%}'.format(null_ratio))

Доля пропусков в столбце days_employed составляет 10%


In [17]:
# доля пропущенных значений в столбце 'total_income' от общего числа значений
null_ratio = df['total_income'].isna().sum() / df.shape[0]
print('Доля пропусков в столбце total_income составляет {:.0%}'.format(null_ratio))

Доля пропусков в столбце total_income составляет 10%


Итак, доля пропусков в каждом из столбцов составляет по 10%. Это критичный показатель, такие пропуски в значениях повредят расчётам.

Возможные причины пропусков:
* Клиенты, оформлявшие кредит, выбрали способ получения кредита без предоставления справки о стаже и доходах. Тогда такие данные в базу не вносились.
* Данные могли быть утеряны по техническим причинам, например из-за сбоя при выгрузке файла.

Данные в столбцах `days_employed` и `total_income` являются количественными, пропуски в таких переменных заполняются характерными значениями - средним арифметическим либо медианой.

Показатели ежемесячного дохода и трудового стажа имеют большой разброс, в таком случае для заполнения пропусков лучше воспользоваться медианным значением.

In [18]:
# находим медианное значение по столбцу 'days_employed' и заполняем им пропуски
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

# находим медианное значение по столбцу 'total_income' и заполняем им пропуски
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)

Проверяем, что в таблице не осталось пропущенных значений.

In [19]:
# подсчет пропущенных значений
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.3 Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный, с помощью метода astype() для более понятного отображения и выведем первые 10 строк.

In [20]:
# заменяем тип данных в столбце 'total_income' на int и выводим первые 10 строк таблицы
df['total_income'] = df['total_income'].astype('int')
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Тип данных в столбце `total_income` изменен.

### 2.4 Дубликаты

Подсчитаем количество явных дубликатов.

In [21]:
# подсчёт явных дубликатов
print('Количество явных дубликатов:', df.duplicated().sum())

Количество явных дубликатов: 55


Удалим явные дубликаты и снова сделаем подсчёт.

In [22]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

# проверка на отсутствие дубликатов
print('Количество явных дубликатов:', df.duplicated().sum())

Количество явных дубликатов: 0


Теперь избавимся от неявных дубликатов. Для этого выведем на экран список уникальных значений столбца `education`, отсортированный в алфавитном порядке

In [23]:
# просмотр уникальных значений уровня образования
sorted(df['education'].unique())

['ВЫСШЕЕ',
 'Высшее',
 'НАЧАЛЬНОЕ',
 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
 'Начальное',
 'Неоконченное высшее',
 'СРЕДНЕЕ',
 'Среднее',
 'УЧЕНАЯ СТЕПЕНЬ',
 'Ученая степень',
 'высшее',
 'начальное',
 'неоконченное высшее',
 'среднее',
 'ученая степень']

Все дубликаты в данном столбце появились по причине использования разного регистра. Приведем все значения к нижнему регистру и проверим остались ли дубликаты.

In [24]:
# приводим значения к нижнему регистру
df['education'] = df['education'].str.lower()

# проверка уникальных значений
sorted(df['education'].unique())

['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']

Дубликаты по колонке `education` устранены.

---

Проверим дубликаты по колонке `family_status`, для этого также выведем на экран уникальные значения по столбцу.

In [25]:
# просмотр уникальных значений семейного статуса
sorted(df['family_status'].unique())

['Не женат / не замужем',
 'в разводе',
 'вдовец / вдова',
 'гражданский брак',
 'женат / замужем']

Первое значение начинается с буквы верхнего регистра, приведем список к единообразию - нижнему регистру.

In [26]:
# приводим значения к нижнему регистру
df['family_status'] = df['family_status'].str.lower()

# проверка уникальных значений
sorted(df['family_status'].unique())

['в разводе',
 'вдовец / вдова',
 'гражданский брак',
 'женат / замужем',
 'не женат / не замужем']

Неявных дубликатов по колонке `family_status` не осталось, обработка завершена.

---

Проверим дубликаты по колонке `income_type`.

In [27]:
# просмотр уникальных значений типа занятости
sorted(df['income_type'].unique())

['безработный',
 'в декрете',
 'госслужащий',
 'компаньон',
 'пенсионер',
 'предприниматель',
 'сотрудник',
 'студент']

Неявных дубликатов по колонке `income_type` нет, обработка не требуется.

### 2.5 Категоризация данных

Для удобства храрения и обращения с информацией создадим отдельные «словари», где названию категории будет соответствовать номер. И в будущих таблицах будем обращаться уже не к длинной строке, а к её числовому обозначению.

Создадим словари для категорий `education` и `family_status` и в исходной таблице оставим только их идентификаторы.

In [28]:
# создаем словарь для категории 'education' и выводим его на экран
education_dict = df[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict.sort_values(by='education_id'))

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень


In [29]:
# создаем словарь для категории 'family_status' и выводим его на экран
family_status_dict = df[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.sort_values(by='family_status_id'))

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  не женат / не замужем


In [30]:
# удаляем столбцы 'education' и 'family_status' и выводим на экран первые 10 строк таблицы
del df['education']
del df['family_status']
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


---

Создадим столбец `total_income_category` с категориями на основании следующих диапазонов:
* 0–30000 — `'E'`;
* 30001–50000 — `'D'`;
* 50001–200000 — `'C'`;
* 200001–1000000 — `'B'`;
* 1000001 и выше — `'A'`.

Создадим функцию `total_income_category()`, которая оценит значения столбца `total_income` и присвоит им категории, в соответствии с диапазоном выше. Добавим эти категории в столбец `total_income_category` и выведем на экран первые 10 строк таблицы.

In [31]:
# функция оценки категории дохода
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    elif total_income <= 50000:
        return 'D'
    elif total_income <= 200000:
        return 'C'
    elif total_income <= 1000000:
        return 'B'
    return 'A'
# добавление нового столбца 'total_income_category' с помощью функции total_income_category()
df.insert(9, 'total_income_category', df['total_income'].apply(total_income_category))
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,B,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,C,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,C,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,B,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,C,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,B,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,B,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,C,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,C,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,C,покупка жилья для семьи


---

Создадим столбец `purpose_category`, в который войдут следующие категории:
* `'операции с автомобилем'`,
* `'операции с недвижимостью'`,
* `'проведение свадьбы'`,
* `'получение образования'`.

Изучим значения столбца `purpose` с помощью метода `unique()`.

In [32]:
# просмотр уникальных значений цели получения кредита
sorted(df['purpose'].unique())

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

Создадим функцию `purpose_category()`, которая оценит значения столбца `purpose` и присвоит им категории, в соответствии со списком выше. Добавим эти категории в столбец `purpose_category` и выведем на экран первые 10 строк таблицы.

In [33]:
# функция определения категории цели кредита
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образов' in purpose:
        return 'получение образования'
    return 'операции с недвижимостью'

# добавление нового столбца 'purpose_category' с помощью функции purpose_category()
df['purpose_category'] = df['purpose'].apply(purpose_category)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,B,покупка жилья,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,C,приобретение автомобиля,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,C,покупка жилья,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,B,дополнительное образование,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,C,сыграть свадьбу,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,B,покупка жилья,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,B,операции с жильем,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,C,образование,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,C,на проведение свадьбы,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,C,покупка жилья для семьи,операции с недвижимостью


**Выводы**

Предобработка обнаружила четыре проблемы в данных:

- аномальные значения,
- пропущенные значения,
- некорректный тип данных,
- дубликаты — явные и неявные.

Мы проверили и скорректировали аномальные значения. Пропущенные значения мы заменили на медианные. А также исправили тип данных в колонке `total_income` с вещественного на целочисленный.

Без дубликатов исследование станет более точным. Для определения неявных дубликатов использовался метод `unique()`, чтобы наглядно оценить возможные дубликаты. Например, название одного и того же уровня образования может быть записано немного по-разному. Наиболее вероятная причина появления таких дубликатов - человеческий фактор.

Также была проведена категоризация данных для упрощения работы с таблицей.

Теперь можно перейти к ответам на вопросы. 

## 3. Ответы на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Сгруппируем данные о невозврате кредитов по количеству детей у заемщика и вычислим процентное соотношение невозвратов по категориям.

In [34]:
# группируем данные по количеству детей
children_correlation = df.pivot_table(index='children', values='debt', aggfunc=['sum', 'count'])
children_correlation.columns = ['debt', 'count']

# вычисляем процентое соотношение невозвратов по категориям
children_correlation['ratio'] = children_correlation['debt'] / children_correlation['count'] * 100
display(children_correlation)

,debt,count,ratio
children,,,
0,1063,14106,7.535800
1,445,4856,9.163921
2,202,2128,9.492481
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


На первом месте по невозвратам кредитов находятся семьи с 4-мя детьми (9.75% невозвратов), на втором месте семьи с 2-мя детьми (9.49% невозвратов). Семьи с 5 детьми всегда возвращали кредиты в срок.

При таких результатах, мы не можем сказать, что существует взаимосвязь между количеством детей и возвратом кредита в срок.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем данные о невозврате кредитов по семейному статусу заемщика и вычислим процентное соотношение невозвратов по категориям.

In [35]:
# группируем данные по семейному статусу
family_status_correlation = df.pivot_table(index='family_status_id', values='debt', aggfunc=['sum', 'count'])
family_status_correlation.columns = ['debt', 'count']

# вычисляем процентое соотношение невозвратов по категориям
family_status_correlation['ratio'] = family_status_correlation['debt'] / family_status_correlation['count'] * 100
display(family_status_correlation)
display(family_status_dict)

,debt,count,ratio
family_status_id,,,
0,931,12344,7.542126
1,388,4162,9.322441
2,63,959,6.569343
3,85,1195,7.112971
4,274,2810,9.750890


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


На первом месте по невозвратам кредитов находятся неженатые / незамужние заемщики - 9.75% невозвратов. На втором месте - заемщики, находящиеся в гражданском браке (9.32%). Далее - женатые (7.54%), в разводе и вдовствующие - 7.11% и 6.57% соответственно.

Таким образом, четкую взаимосвязь между семейным статусом и возвратом кредита в срок мы не можем.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
# группируем данные по уровню дохода
total_income_correlation = df.pivot_table(index='total_income_category', values='debt', aggfunc=['sum', 'count'])
total_income_correlation.columns = ['debt', 'count']

# вычисляем процентое соотношение невозвратов по категориям
total_income_correlation['ratio'] = total_income_correlation['debt'] / total_income_correlation['count'] * 100
total_income_correlation

,debt,count,ratio
total_income_category,,,
A,2,25,8.000000
B,356,5041,7.062091
C,1360,16032,8.483034
D,21,350,6.000000
E,2,22,9.090909


Наибольшая доля невозвратов (9.09%) приходится на людей с низким уровнем дохода. 8.48% приходится на людей со средним уровнем дохода. На третьем месте люди с высоким уровнем дохода (8%), далее выше среднего (8.06%) и ниже среднего (6%).

При таких результатах, мы не можем сказать, что существует взаимосвязь между уровнем дохода и возвратом кредита в срок.

### Как разные цели кредита влияют на его возврат в срок?

In [37]:
# группируем данные по уровню дохода
purpose_correlation = df.pivot_table(index='purpose_category', values='debt', aggfunc=['sum', 'count'])
purpose_correlation.columns = ['debt', 'count']

# вычисляем процентое соотношение невозвратов по категориям
purpose_correlation['ratio'] = purpose_correlation['debt'] / purpose_correlation['count'] * 100
purpose_correlation

,debt,count,ratio
purpose_category,,,
операции с автомобилем,403,4308,9.354689
операции с недвижимостью,782,10814,7.231367
получение образования,370,4014,9.217738
проведение свадьбы,186,2334,7.969152


Распределение невозвратов кредита по целям кредитования следующее:
1. Операции с автомобилем - 9.35%
2. Получение образования - 9.21%
3. Проведение свадьбы - 7.97%
4. Операции с недвижимостью - 7.23%



Наибольшая доля приходится на операции с автомобилем - 9.35%, наименьшая - на операции с недвижимостью (7.23%).

## 4. Итоги исследования

Исследование было проведено для определения влияния разных факторов на возврат заемщиками кредитов в срок. Данные предоставлены кредитным отделом банка. В ходе их обзора выявились проблемы с аномалиями, пропусками и дубликатами.

На этапе предобработки были устранены следующие аномалии:
* Отрицательные значения стажа и количества детей взяты по модулю;
* Неправдоподобно большие значения стажа по предположению внесены в часах, вместо дней. Поэтому такие значения переведены в дни;
* Выдающееся значение детей в семье - 20 исправлено на 2;
* Возраст заемщиков 0 лет исправлен на медианное значение.

Пропуски значений по столбцам `days_employed` и `total_income`  были заполнены медианными значениями, так как являются количественными характеристиками.

Были устранены неявные дубликаты по столбцу `education`. Данные в таблице были категоризированы, а также приведены к необходимым типам в соответствии с наименованием столбцов и содержанием.

В ходе исследования были даны ответы на четыре вопроса.

 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
    
    *Однозначной взаимосвязи между количеством детей и невозвратом кредита в срок не обнаружено.*
   

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

    *Зависимость от семейного положения таже не обнаружена.*

 
 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

    *Взаимосвязь между уровнем дохода и возвратом кредита в срок не подтвердилась.*

 
 4. Как разные цели кредита влияют на его возврат в срок?

    *Наибольшая доля невозвратов в срок приходится на операции с автомобилями. Меньше всего риск невозвратов в срок при выдаче кредита на операции с недвижимостью.*